# To be continue...

In [ ]:
!pip install numba
!pip install numpy
!pip install matplotlib

In [1]:
from numba import vectorize, njit
import numpy as np

#Due to the interactive term, we need to add it:
#But the escence of the code remains the same

@njit
def stability_criteria_of_dt(V, dx):
    Vmax = max(V)

    numerator = -2 * dx * dx
    denominator = -2 + dx * dx * Vmax

    dt = numerator / denominator 

    order = np.floor(np.log10(dt))
    # Scale and round down to one significant figure
    dt = np.floor(dt / (10** order)) * (10** order)
    
    return dt * 1e-1

@vectorize
def hamiltonian_BEC(fxm1, fx, fxp1, V, U0 ,dx):
  lap = (fxm1 + fxp1 - 2 * fx) / (dx * dx)
  Kterm = -0.5 * lap
  Vterm = V * fx
  Nonlin_term = U0 * (fx * fx) * fx
  return Kterm + Vterm + Nonlin_term

@vectorize
def RnewBEC(Rold, Itm1, It, Itp1, V, U0, dt, dx):
  nd_term = dt * hamiltonian_BEC(Itm1, It, Itp1, V, U0, dx)

  numerator = Rold + nd_term
  denominator = 1 - dt * U0 * Rold * It
  return numerator / denominator

@vectorize
def InewBEC(Iold, Rtm1, Rt, Rtp1, V, U0, dt, dx):
  nd_term = dt * hamiltonian_BEC(Rtm1, Rt, Rtp1, V, U0, dx)

  numerator = Iold - nd_term

  denominator = 1 + dt * U0 * Iold * Rt
  return numerator / denominator

@njit
def rolled_vector(f):
  fx1 = f
  fxplus1 = np.roll(f, 1)
  fxplus1[0] = 0

  fxminus1 = np.roll(f, -1)
  fxminus1[-1] = 0
  return fxminus1, fx1, fxplus1

In [2]:
def Interacting_simulation(L, dx, Ntimesteps):
   
    X = np.arange(-L, L, dx) #the discretized X-space
    V = np.zeros_like(X) # we set the external potential to 0
    U0 = np.full(len(X), -2) #and the interacting term to -2

    #given the potential and the dx
    #we generate our stable dt
    dt = stability_criteria_of_dt(V, dx) 
    tfinal = Ntimesteps * dt
    T = np.arange(0, tfinal, dt * 0.5) # our discretized timespace, by jumps of dt/2

    #part b) Initial conditions:
    R = 1 / (np.sqrt(2) * np.cosh(X))
    I = np.zeros_like(R)

    #2nd step: the first half step of the simulation:
    Rm1, R, Rp1 = rolled_vector(R)
    I = I - hamiltonian_BEC(Rm1, R, Rp1, V, U0, dx) * dt * 0.5

    #Now we generate the lists to store our data:
    Prob_list = np.zeros(shape= ( len(T), len(X)))
    R_list = np.zeros_like(Prob_list)
    I_list = np.zeros_like(Prob_list)

    #and add the initial coonditions
    Prob_list[0][:] = R * R
    R_list[0][:] = R
    I_list[0][:] = I

    #now we begin the iterations, how we are making increments of half
    #steps, our T = [0, dt/2, dt, 3dt/2, ...] and we already do the first one,
    #we will start from T[2:] ...
    for i in range(1, len(T[2:])):
       
       #if the time is integer:
        if i % 2 == 0:
            Im1, I, Ip1 = rolled_vector(I)
            Rstep = RnewBEC(R, Im1, I, Ip1, V, U0,dt, dx)
            
            #we compute the Probability for integer time
            prob = Rstep * R + I * I
            R = Rstep

        #if our time is half integer:
        else:
            Rm1, R, Rp1 = rolled_vector(R)
            Istep = InewBEC(I, Rm1, R, Rp1, V, U0, dt, dx)
            
            #we compute the Probability for half integer
            prob = Istep * I + R * R
            I = Istep

        #we save the data
        Prob_list[i+1][:] = prob
        R_list[i][:] = R
        I_list[i][:] = I


    return [Prob_list, R_list, I_list]

In [3]:
#parameters of the simulation, feel free to change it!
L = 12
dx = 0.1
Ntimesteps = 2 ** 14

ydatac = Interacting_simulation(L, dx, Ntimesteps)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rcParams

# Aumenta el límite de tamaño de la animación incrustada a, por ejemplo, 100 MB
rcParams['animation.embed_limit'] = 50

X = np.arange(-L, L, dx) #the discretized X-space
 # we generate our armonic potential

#here in order to plot more smoothly,
#we will only consider frames

data = ydatac
data1 = data[0][::5]
data2 = data[1][::5]
data3 = data[2][::5]


x_values =  X# Los valores en el eje X (espacio)

# 2. Crear la figura y un solo eje
fig, ax = plt.subplots(figsize=(6, 4))  # Solo un gráfico

# Inicialmente graficamos los primeros frames de los tres conjuntos de datos
line1, = ax.plot(x_values, data1[0], color='b', label= r'$|\psi|²$')
line2, = ax.plot(x_values, data2[0], color='r', label= r'$\Re(\psi)$')
line3, = ax.plot(x_values, data3[0], color='g', label= r'$\Im(\psi)$' )

# 3. Ajustamos los ejes, agregamos etiquetas, título y grid
ax.set_xlim(-L, L)
ax.set_ylim(-1, 1)
ax.set_xlabel('X')
ax.set_ylabel('Amplitude ')
ax.set_title('Time evolution of the system')
ax.grid(True)
ax.legend(loc= 4)  # Mostrar la leyenda para las tres funciones

# Añadir texto para mostrar el instante de tiempo
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

# 4. Función de actualización para la animación
def update(frame):
    line1.set_ydata(data1[frame])  # Actualizamos los datos de la primera línea
    line2.set_ydata(data2[frame])  # Actualizamos los datos de la segunda línea
    line3.set_ydata(data3[frame])  # Actualizamos los datos de la tercera línea

    time_text.set_text(f'iteration: {frame}')  # Actualizar el texto del tiempo

    return line1, line2, line3, time_text  # Retornamos el texto junto con las líneas

# 5. Crear la animación
ani = FuncAnimation(fig, update, frames= int(Ntimesteps / 5), blit=True)

# 6. Mostrar la animación en Google Colab
from IPython.display import HTML
HTML(ani.to_jshtml())

And as we can observe here, the wavefunctions probability in time, remains the same. At first glance, this can be strange, but then by remembering the lectures of professor Emanuel Henn, the initial wavefunction is, in fact, a soliton solution for the interacting term when the $U_0$ parameter is negative, which corresponds to an eigenfunction of the operator of the Non linear Schrödinger Equation!